In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

# Load data

In [2]:
data = pd.read_csv("Doyle_Christie_dataset/train.csv")
data = data.drop(columns=['Unnamed: 0',])

In [3]:
test_data = pd.read_csv("Doyle_Christie_dataset/test.csv")
test_data = test_data.drop(columns=['Unnamed: 0',])
test_data['labels'] = test_data['author'].apply(lambda x: 0 if x == 'Doyle' else 1)

In [4]:
data['labels'] = data['author'].apply(lambda x: 0 if x == 'Doyle' else 1)

# Functions

In [8]:
def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description):
    model = LogisticRegression().fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Logreg + ', description)
    print('Test Score: ', score)
    print()
    return model

def svm_classify(X_tr, y_tr, X_test, y_test, description):
    model = SVC().fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('SVM + ', description)
    print('Test Score: ', score)
    print()
    return model

In [10]:
def main(data, test_data):
    bow_vect = CountVectorizer()
    bow_train = bow_vect.fit_transform(data['sentence'])
    bow_test = bow_vect.transform(test_data['sentence'])
    
    bow_model = simple_logistic_classify(X_tr = bow_train, 
                                     y_tr = data['labels'],
                                     X_test = bow_test, 
                                     y_test = test_data['labels'], 
                                     description='bag of words')
    
    tfidf_vect = TfidfVectorizer()
    tfidf_train = tfidf_vect.fit_transform(data['sentence'])
    tfidf_test = tfidf_vect.transform(test_data['sentence'])
    
    tfidf_model = simple_logistic_classify(X_tr = tfidf_train, 
                                       y_tr = data['labels'],
                                       X_test = tfidf_test,
                                       y_test = test_data['labels'], 
                                       description='tf-idf')
    
    svm_model = svm_classify(X_tr = tfidf_train, 
                           y_tr = data['labels'],
                           X_test = tfidf_test,
                           y_test = test_data['labels'], 
                            description='tf-idf')

In [11]:
main(data=data, test_data=test_data)

/home/antonauna/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logreg +  bag of words
Test Score:  0.7415

Logreg +  tf-idf
Test Score:  0.7455

SVM +  tf-idf
Test Score:  0.757

